# MIATT Final Exam May 8, 2019  3:00pm - 5:00pm

## Problem Background Statement

A set of many thousands of data sets have been collected and pre-processed for analysis of the human brain.  During the intital processing the Anterior Commisure (AC) landmark was used to align all the data sets such the AC point was at physical location $(0.0, 0.0, 0.0)$.

A new researcher is interested in studying the Cerebellum (a smaller part of the brain posterior to the brain stem). The Forth Ventricle Notch (VN4) of the Cerebellum is a primary landmark point that has been identified in all the datasets.

<img src="4thVentricleNotchOfCerebellum.png" style="width:700px"/><br><br>

## Problem Task Statement

In order to facilitate the new research, you must resample all datasets so that VN4 landarmark point of the cerebellum is represented in an image with the following properties:

1. The VN4 is located at exactly $(+0.0, +60.0, -30.0)$ in the ITK LPS coordinate system.
1. The VN4 location is in the center of the voxel lattice
1. The final cerebellum image voxel samples are isotropic voxel spacing (at a user specified value)
1. The final cerebellum image voxel size is 100mm in the inferior/supperior direction
1. The final cerebellum image voxel size is 150mm in the left/right direction
1. The final cerebellum image voxel size is 90mm in the anterior/posterior direction
1. The final cerebellum image must have a direction cosign that is an identity transform




## NOTE: Many of these hints will *NOT* be used in your solution:
### HINTS:
- Resample Hints
    - https://itk.org/ITKExamples/src/Filtering/ImageGrid/ResampleAnImage/Documentation.html?highlight=resample
- Transform Hints
    - https://itk.org/Doxygen/html/classitk_1_1Euler2DTransform.html
    - https://itk.org/Doxygen/html/classitk_1_1ThinPlateSplineKernelTransform.html
        - `itk.ThinPlateSplineKernelTransform.GetTypes()`
        - `my_tps_transform = itk.ThinPlateSplineKernelTransform[itk.ctype('double'),2]`
    - https://itk.org/Doxygen/html/classitk_1_1TranslationTransform.html



In [67]:
# Boilerplate module inclusions
import itk
import os
import numpy as np
import pandas as pd
import pprint
from matplotlib import pyplot as plt
%matplotlib inline
from ipywidgets import interact, fixed
from ipywidgets import FloatSlider
from itkwidgets import view
from os.path import join

In [68]:
def read_fiducial_points(fcsv_file: str, fiducial_names:list) -> dict:
    """
    This function is used to read Slicer landmark files and extract landmark points
    :param: fcsv_file, a string representing the filename of the fcsv file to read
    :param: fiducial_names, a list of strings with the names of the fiducials you wish to extract.
    :return: A dictionary with keys as the fiducial_names and values as the list of ITK
      coordinates in millimeter space.
    """
    col_names="id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID"
    with open(fcsv_file, 'r') as fid: 
        df = pd.read_csv(fid, sep=',', comment='#', names=col_names.split(',')) 

    fid_pts = {}
    for fid in fiducial_names:
        fid_pts[fid] = df[ df['label'] == fid].iloc[0][1:4].values
        # Flip RAS Slicer fiducial locations to be ITK compliant LPS fiducials in floating point format
        fid_pts[fid][0] = -1*fid_pts[fid][0]
        fid_pts[fid][1] = -1*fid_pts[fid][1]
        fid_pts[fid] = fid_pts[fid].astype(np.float64)
    return fid_pts

In [91]:
class FinalExamSubjectInfo():
    """
    A class to assist with identifying files for a subject.
    Example:
       subj1 = FinalExamSubjectInfo("0123")
       lmkfilename = subj1.get_slicer_landmark_filename()
       imgfilename = subj1.get_t1_image_filename()
       image = itk.imread(imgfilename)
    """
    def __init__(self, subject_id: str) -> None:
        """
        Constructor that accepts a subject identifier as a string (i.e. "0123")
        and sets initial state variables.
        """
        self.input_data_dir = '/nfsscratch/opt/ece5490/data/MIATT_EYES'
        self.output_data_dir = './'  # Output data dir is in the current repository
        self.subject_id = subject_id
    def get_subject_id(self) -> str:
        return self.subject_id
    
    def get_t1_image_filename(self) -> str:
        """
        :return: Returns a string representing this subjects t1 image path
        """
        t1_image_filename = os.path.join(self.input_data_dir,
                                         self.subject_id,
                                         self.subject_id + "_hcpy1",
                                         'TissueClassify',
                                         't1_average_BRAINSABC.nii.gz')
        return t1_image_filename
    def get_slicer_landmark_filename(self) -> str:
        """
        :return: Returns a string representing this subjects landmark file path
        """
        slicer_landmarks = os.path.join(self.input_data_dir,
                                     self.subject_id,
                                     self.subject_id + "_hcpy1",
                                     'ACPCAlign',
                                     'BCD_ACPC_Landmarks.fcsv')
        return slicer_landmarks
    def get_output_cerebellum_filename(self, current_iso_spacing: float) -> str:
        """
        :return: Returns the string representing the desired output cerebellum file path
        """
        cerebellum_image_filename = os.path.join(self.output_data_dir,
                                         self.subject_id + "_cerebellum_final_{0:0.2f}".format(current_iso_spacing)+ ".nii.gz")
        return cerebellum_image_filename

In [95]:
PixelType = itk.ctype('float')          # read as float and recast to char later
ScalarType = itk.ctype('double')        #TODO: This *may* be wrong you might need to change it
Dimension = 3                           

DirectionType =  itk.Matrix[ScalarType,Dimension,Dimension]
ImageType = itk.Image[PixelType, Dimension]
ReaderType = itk.ImageFileReader[ImageType]

interpolatorType = itk.LinearInterpolateImageFunction[ImageType, ScalarType]
interpolator = interpolatorType.New()

def create_cerebellum_image(current_subject_object : FinalExamSubjectInfo, required_iso_spacing: float ) -> ImageType:
    """
    This function takes in a subject object and generates a cerebellum image
    with the requested isotropic voxel spacing size.
    :param: current_subject_object
    :param: required_iso_spacing - The final cerebellum image voxel samples are isotropic voxel spacing
    """
    imgfilename = current_subject_object.get_t1_image_filename()
    reader = ReaderType.New()
    reader.SetFileName(imgfilename)  # set the t1 img file name to read
    reader.Update()  # update the reader object
    # The following is commented out
    inputImage = reader.GetOutput()
    
    important_landmarks = ['AC','PC','VN4', 'LE', 'RE']
    lmkfilename = current_subject_object.get_slicer_landmark_filename()
    landmarks = read_fiducial_points(fcsv_file=lmkfilename, fiducial_names=important_landmarks)  # read in the fiducial we are interested in
    # The VN4 is located within 1mm of (+0.0,+60.0,−30.0) in the ITK LPS coordinate system.
    required_center_pixel = np.array([+0.0,+60.0,-30.0])
    required_spacing = np.array([required_iso_spacing]*Dimension) #TODO: This *may* be wrong you might need to change it
    # The final cerebellum image voxel size is 100mm in the superior/inferior direction
    required_Superior_Inferior_length = 100
    # The final cerebellum image voxel size is 150mm in the left/right direction
    required_Left_Right_length = 150
    # The final cerebellum image voxel size is 90mm in the posterior/anterior direction
    required_Posterior_Anterior_length = 90
    # TODO: Compute the number of voxels in the image.  Use a formula instead of hardcoding values\
    required_size = [(required_Left_Right_length/required_iso_spacing), 
         (required_Posterior_Anterior_length/required_iso_spacing), 
         (required_Superior_Inferior_length/required_iso_spacing)]
    
    # The final cerebellum image must have a direction cosine that is an identity transform
    required_direction_cosine = DirectionType()
    required_direction_cosine.SetIdentity()
    
    # The VN4 location is in the center of the voxel lattice
    required_origin = [required_center_pixel[0]-(required_Left_Right_length/2), 
                       required_center_pixel[1]-(required_Posterior_Anterior_length/2), 
                       required_center_pixel[2]-(required_Superior_Inferior_length/2)] 
    
    tx_offset = landmarks["VN4"] - required_center_pixel #TODO: This *may* be wrong you might need to change it

    transform = itk.TranslationTransform[ScalarType, Dimension].New() #TODO: This *may* be wrong you might need to change it
    transform.SetOffset( tx_offset )
    
    ResamplerType = itk.ResampleImageFilter[ImageType, ImageType] #TODO: This *may* be wrong you might need to change it
    resampler = ResamplerType.New()
    
    # interpolator
    interpolator = itk.LinearInterpolateImageFunction[ImageType, itk.D].New()
   
   #TODO: This *may* be wrong you might need to change it
    resampler.SetInput(inputImage)  
    resampler.SetTransform(transform)
    resampler.SetInterpolator(interpolator)
    resampler.SetSize([int(x) for x in required_size])
    resampler.SetOutputSpacing(required_spacing)
    resampler.SetOutputOrigin([float(x) for x in required_origin])
    resampler.SetOutputDirection(required_direction_cosine)

    print("FINAL IMAGE INFORMATION: {0} at {1}".format( current_subject_object.get_subject_id() ,required_iso_spacing ))
    print("Required Size: {0}".format(required_size))
    print("Required Spacing: {0}".format(required_spacing))
    print("Required Origin: {0}".format(required_origin))

    resampler.Update()
    outputImage = resampler.GetOutput() #TODO: This *may* be wrong you might need to change it\
    return outputImage

In [96]:
## Now make images for subjects 0547 and 0298 at isotropic voxel resolutions of 0.5 and 2.0

output_image_list = list()
for current_subject in ["0547", "0298"]:
    for current_iso_spacing in [ 0.5, 2.0 ]:
        subj1 = FinalExamSubjectInfo(current_subject)
        outputImage = create_cerebellum_image(subj1, current_iso_spacing)
        itk.imwrite(outputImage,subj1.get_output_cerebellum_filename(current_iso_spacing))
        output_image_list.append(outputImage)


FINAL IMAGE INFORMATION: 0547 at 0.5
Required Size: [300.0, 180.0, 200.0]
Required Spacing: [0.5 0.5 0.5]
Required Origin: [-75.0, 15.0, -80.0]
FINAL IMAGE INFORMATION: 0547 at 2.0
Required Size: [75.0, 45.0, 50.0]
Required Spacing: [2. 2. 2.]
Required Origin: [-75.0, 15.0, -80.0]
FINAL IMAGE INFORMATION: 0298 at 0.5
Required Size: [300.0, 180.0, 200.0]
Required Spacing: [0.5 0.5 0.5]
Required Origin: [-75.0, 15.0, -80.0]
FINAL IMAGE INFORMATION: 0298 at 2.0
Required Size: [75.0, 45.0, 50.0]
Required Spacing: [2. 2. 2.]
Required Origin: [-75.0, 15.0, -80.0]


In [98]:
## Utility to view outputs of the four images
view(output_image_list[0],
    ui_collapsed=True,
    annotations=True,
    interpolation=False,
    cmap='Grayscale',
    mode='x',
    shadow=True,
    slicing_planes=False,
    gradient_opacity=0.22,)

Viewer(cmap='Grayscale', gradient_opacity=0.22, interpolation=False, mode='x', rendered_image=<itkImagePython.…